在[3. Facial Keypoint Detection, Complete Pipeline.ipynb]()中可以发现这里奥巴马夫妇的脸并没有完全包括到矩形区域里面，如果直接将矩形区域传入模型，那么预测的关键点

~~**下巴的轮廓将和下嘴唇非常靠近！**~~

实际上是，整个预测关键点的scatter图像感觉向中心缩小了！主要是因为，**训练数据基本没有这种haar_cascade输出的这种图像——绝大部分区域的都是人脸**，所以

需要增大输入模型的矩形区域。对于haar_cascade得到的人脸区域，左右上下个增加`60`个像素点，或者将宽高扩大两倍。

---

而在[4. Fun with Keypoints.ipynb]()中用到以下的旋转函数，

In [1]:
def rotate_image(mat, angle):
    height, width = mat.shape[:2]
    image_center = (width / 2, height / 2)

    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1)

    radians = math.radians(angle)
    sin = math.sin(radians)
    cos = math.cos(radians)
    bound_w = int((height * abs(sin)) + (width * abs(cos)))
    bound_h = int((height * abs(cos)) + (width * abs(sin)))

    # 之前图片的旋转中心（图片中心点）在image_center,在新的图片的中心点为(bound_w / 2, bound_h / 2)
    # 也就是说中心点相距图片的左上角（(0, 0)）发生了移动
    # 相减即为中心点移动向量，而图片中的其它点的移动向量也是这个
    rotation_mat[0, 2] += ((bound_w / 2) - image_center[0])
    rotation_mat[1, 2] += ((bound_h / 2) - image_center[1])

    rotated_mat = cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h))
    return rotated_mat

但是实现的旋转的效果并不理想，通过opencv画的关键点发现CNN模型预测的关键点基本没有随着脸的偏转而发生对应的偏转。

同样，训练数据中人脸有足够倾斜的图片较少，模型可能没有学习这方面的参数。